In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf

In [2]:
def load_data():
    circle_data = np.load('/Users/shaily/Desktop/QuickDraw/circle.npy')
    triangle_data = np.load('/Users/shaily/Desktop/QuickDraw/triangle.npy')
    train_c_f = circle_data[0:90000, :]
    train_t_f = triangle_data[0:90000, :]
    train_c_f = train_c_f.reshape((90000, 28, 28))
    train_t_f = train_t_f.reshape((90000, 28, 28))
    train_c_l = np.zeros(len(train_c_f))
    train_t_l = np.ones(len(train_t_f))
    train_labels=np.append(train_c_l,train_t_l,axis=0)
    train_features=np.append(train_c_f,train_t_f,axis=0)
   
    test_c_f = circle_data[90001:100001, :]
    test_t_f = triangle_data[90001:100001, :]
    test_c_f = test_c_f.reshape((10000, 28, 28))
    test_t_f = test_t_f.reshape((10000, 28, 28))
    test_c_l = np.zeros(len(test_c_f))
    test_t_l = np.ones(len(test_t_f))
    test_f_combine = np.append(test_c_f, test_t_f, axis=0)
    test_l_combine = np.append(test_c_l, test_t_l, axis=0)
    
    vtest_c_f = circle_data[100002:110002, :]
    vtest_t_f = triangle_data[100002:110002, :]
    vtest_c_f = vtest_c_f.reshape((10000, 28, 28))
    vtest_t_f = vtest_t_f.reshape((10000, 28, 28))
    vtest_c_l = np.zeros(len(vtest_c_f))
    vtest_t_l = np.ones(len(vtest_t_f))
    vtest_f_combine = np.append(vtest_c_f, vtest_t_f, axis=0)
    vtest_l_combine = np.append(vtest_c_l, vtest_t_l, axis=0)
  
    return train_features, train_labels ,test_f_combine,test_l_combine,vtest_f_combine,vtest_l_combine
train_f,train_l,test_f,test_l,validate_f,validate_l=load_data()

In [3]:
#Preprocessing
image_size = 28
num_labels = 2
num_channels = 1
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.
train_f=(train_f.astype(np.float32) - 
                    pixel_depth / 2) / pixel_depth
test_f=(test_f.astype(np.float32) - 
                    pixel_depth / 2) / pixel_depth
validate_f=(validate_f.astype(np.float32) - 
                    pixel_depth / 2) / pixel_depth
train_f=train_f.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
test_f=test_f.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
validate_f=validate_f.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)

a=[]
for i in range(0,180000):
    if(i<90000):
        a.append([1,0])
    else:
        a.append([0,1])
train_l=np.array(a).astype(np.float32)
a=[]
for i in range(0,20000):
    if(i<10000):
        a.append([1,0])
    else:
        a.append([0,1])
test_l=np.array(a).astype(np.float32)
a=[]
for i in range(0,20000):
    if(i<10000):
        a.append([1,0])
    else:
        a.append([0,1])
validate_l=np.array(a).astype(np.float32)

In [4]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_f, train_l = randomize(train_f, train_l)
test_f, test_l = randomize(test_f, test_l)
validate_f, validate_l = randomize(validate_f, validate_l)

In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [6]:
#Model
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(validate_f)
  tf_test_dataset = tf.constant(test_f)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    hidden=tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding='SAME',data_format="NHWC")
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    hidden=tf.nn.max_pool(hidden,strides=[1,2,2,1],ksize=[1,2,2,1],padding='SAME',data_format="NHWC")
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
#Training and Evaluation
num_steps = 1001
with tf.Session(graph=graph) as session:

  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_l.shape[0] - batch_size)
    batch_data = train_f[offset:(offset + batch_size), :, :, :]
    batch_labels = train_l[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)

    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), validate_l))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_l))


Initialized
Minibatch loss at step 0: 0.741833
Minibatch accuracy: 50.0%
Validation accuracy: 50.0%
Minibatch loss at step 50: 0.097265
Minibatch accuracy: 100.0%
Validation accuracy: 96.1%
Minibatch loss at step 100: 0.012824
Minibatch accuracy: 100.0%
Validation accuracy: 96.3%
Minibatch loss at step 150: 0.045139
Minibatch accuracy: 100.0%
Validation accuracy: 97.5%
Minibatch loss at step 200: 0.036502
Minibatch accuracy: 100.0%
Validation accuracy: 97.7%
Minibatch loss at step 250: 0.012025
Minibatch accuracy: 100.0%
Validation accuracy: 97.4%
Minibatch loss at step 300: 0.006965
Minibatch accuracy: 100.0%
Validation accuracy: 97.3%
Minibatch loss at step 350: 0.037725
Minibatch accuracy: 100.0%
Validation accuracy: 97.7%
Minibatch loss at step 400: 0.183316
Minibatch accuracy: 93.8%
Validation accuracy: 97.9%
Minibatch loss at step 450: 0.065870
Minibatch accuracy: 93.8%
Validation accuracy: 97.9%
Minibatch loss at step 500: 0.149886
Minibatch accuracy: 93.8%
Validation accuracy: 